<b>The dataset is very unbalanced. </b> For example, C3S4 and C3S4 classes have only one sample. This situation is not good in respect of  using the oversampling methods. <br>

I tried to create a new class (called the other) by combining other classes which have a small sample size.
Finally, there are six classes to classify, named 'C2S1', 'C3S1', 'C3S2', 'C4S1', 'C4S2', 'Other'.


## Results

I have used optima for optimizing the model. The results were not very good. Although our model gained reasonable precision/recall scores in a few classes, some classes have bad scores.<br>

- (Test Set) R2 score : 82.851
- (Test Set) MAE : 0.207831

| class        	| precision 	| recall 	| f1-score 	| support 	|
|--------------	|-----------	|--------	|----------	|---------	|
| 0            	| 1.00      	| 0.97   	| 0.99     	| 76      	|
| 1            	| 0.97      	| 0.99   	| 0.98     	| 204     	|
| 2            	| 0.25      	| 0.17   	| 0.20     	| 6       	|
| 3            	| 0.88      	| 0.88   	| 0.88     	| 26      	|
| 4            	| 0.73      	| 0.67   	| 0.70     	| 12      	|
| 5            	| 0.62      	| 0.62   	| 0.62     	| 8       	|
| accuracy     	|           	|        	| 0.94     	| 332     	|
| macro avg    	| 0.74      	| 0.72   	| 0.73     	| 332     	|
| weighted avg 	| 0.94      	| 0.94   	| 0.94     	| 332     	|

In [45]:
%pip install catboost
%pip install scikit-learn
%pip install seaborn
%pip install numpy
%pip install pandas
%pip install mealpy
%pip install pyswarms
%pip install gdown

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [46]:
# Importing dependencies

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.metrics import r2_score, mean_squared_error, classification_report
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score

from catboost import CatBoostClassifier, Pool

# <span style="color:#e74c3c;"> Reading </span> Data


In [47]:
!gdown 1SaxJ8KPMrV37ZsVTv5F5ava5aXtGRiSn
!gdown 1E6BZ-AEncUOWlsK96SFz5wBK8w7XjmVB
!gdown 195J88Onvr23J8HdFtO3D_Yezi97clcZ9

Downloading...
From: https://drive.google.com/uc?id=1SaxJ8KPMrV37ZsVTv5F5ava5aXtGRiSn
To: d:\projects\python\groundwater\ground_water_quality_2019_post.csv

  0%|          | 0.00/66.7k [00:00<?, ?B/s]
100%|██████████| 66.7k/66.7k [00:00<00:00, 1.80MB/s]
Downloading...
From: https://drive.google.com/uc?id=1E6BZ-AEncUOWlsK96SFz5wBK8w7XjmVB
To: d:\projects\python\groundwater\ground_water_quality_2020_post.csv

  0%|          | 0.00/68.1k [00:00<?, ?B/s]
100%|██████████| 68.1k/68.1k [00:00<00:00, 1.94MB/s]
Downloading...
From: https://drive.google.com/uc?id=195J88Onvr23J8HdFtO3D_Yezi97clcZ9
To: d:\projects\python\groundwater\ground_water_quality_2018_post.csv

  0%|          | 0.00/72.8k [00:00<?, ?B/s]
100%|██████████| 72.8k/72.8k [00:00<00:00, 2.43MB/s]


In [48]:
# Reading data and cleaning, renaming and other data cleaning applications

data1 = pd.read_csv('./ground_water_quality_2018_post.csv')
data2 = pd.read_csv('./ground_water_quality_2019_post.csv')
data3 = pd.read_csv('./ground_water_quality_2020_post.csv')



data2.rename( columns ={ 'EC' : 'E.C', 'CO_-2 ' : 'CO3', 'HCO_ - ' :'HCO3', 'Cl -' : 'Cl',
                        'F -' : 'F', 'NO3- ': 'NO3 ' , 'SO4-2':'SO4' , 'Na+':'Na', 'K+':'K',
                        'Ca+2' : 'Ca', 'Mg+2':'Mg'}, inplace = True)


# dropping redundant columns
data1.drop(['sno','season'], axis = 1, inplace = True)
data2.drop(['sno','season'], axis = 1, inplace = True)
data3.drop(['sno','Unnamed: 8', 'season'], axis = 1, inplace = True)


# creating new columns
data1['year'] = 2018
data2['year'] = 2019
data3['year'] = 2020


# handling and fixing outliers
data3['pH'].iloc[261] = data3['pH'].iloc[261].replace('8..05', '8.05')
data3['pH'] = data3['pH'].apply(pd.to_numeric)

data3['Classification'].iloc[178] = data3['Classification'].iloc[178].replace('O.G', 'OG')
data3['Classification'].iloc[208] = data3['Classification'].iloc[208].replace('O.G', 'OG')


In [49]:
# creating and applying the new_class function

def new_class(X):
    if (X == 'C3S4') | (X == 'C2S2') | (X == 'C4S4') | (X == 'C3S3') | (X == 'C4S3') | (X == 'OG')  | (X == 'C1S1')  :
        return 'Other'
    else:
        return X

data1['Classification'] = data1['Classification'].apply(new_class)
data2['Classification'] = data2['Classification'].apply(new_class)
data3['Classification'] = data3['Classification'].apply(new_class)

data_full = pd.concat([data1, data2, data3], axis = 0)

In [50]:
# total null elements

data_full.isnull().sum()[data_full.isnull().sum() > 0]

gwl     11
CO3    160
dtype: int64

In [51]:
# imputing null values

imp_knn = KNNImputer(n_neighbors=3)

data_full['CO3'] = imp_knn.fit_transform(np.array(data_full['CO3']).reshape(-1,1) )
data_full['gwl'] = imp_knn.fit_transform(np.array(data_full['gwl']).reshape(-1,1) )

In [52]:
data_full.isnull().sum()[data_full.isnull().sum() > 0]

Series([], dtype: int64)

In [53]:
data_full.head()

,district,mandal,village,lat_gis,long_gis,gwl,pH,E.C,TDS,CO3,...,Na,K,Ca,Mg,T.H,SAR,Classification,RSC meq / L,Classification.1,year
0,ADILABAD,Adilabad,Adilabad,19.668300,78.524700,5.09,8.28,745,476.80,0.0,...,49.0,4.0,48.0,38.896,279.934211,1.273328,C2S1,-1.198684,P.S.,2018
1,ADILABAD,Bazarhatnur,Bazarhatnur,19.458888,78.350833,5.10,8.29,921,589.44,0.0,...,42.0,5.0,56.0,63.206,399.893092,0.913166,C3S1,-3.397862,P.S.,2018
2,ADILABAD,Gudihatnoor,Gudihatnoor,19.525555,78.512222,4.98,7.69,510,326.40,0.0,...,45.0,2.0,24.0,38.896,219.934211,1.319284,C2S1,-0.398684,P.S.,2018
3,ADILABAD,Jainath,Jainath,19.730555,78.640000,5.75,8.09,422,270.08,0.0,...,27.0,1.0,32.0,19.448,159.967105,0.928155,C2S1,0.000658,P.S.,2018
4,ADILABAD,Narnoor,Narnoor,19.495665,78.852654,2.15,8.21,2321,1485.44,0.0,...,298.0,5.0,56.0,92.378,519.843750,5.682664,C4S2,-4.396875,P.S.,2018


In [54]:
# creating train data and target

X = data_full.copy()
X.drop('Classification', axis= 1, inplace = True)

y = data_full['Classification']

In [55]:
LB = LabelEncoder()
y = LB.fit_transform(y)
LB.classes_

array(['C2S1', 'C3S1', 'C3S2', 'C4S1', 'C4S2', 'Other'], dtype=object)

In [56]:
# categorical features

cat_feat_idx =  np.where(X.dtypes == 'object')[0]
cat_feat_idx

array([ 0,  1,  2, 22], dtype=int64)

In [57]:
# scaling numerical data

MX = MinMaxScaler()
X.iloc[:, 3:21] = MX.fit_transform(X.iloc[:, 3:21])

In [58]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3 , shuffle = True, stratify=y , random_state= 2)

print(X_train.shape)
print(X_test.shape)

(774, 24)
(332, 24)


In [59]:
# creating class weights

unique_classes = np.unique(y)
weights = compute_class_weight(class_weight='balanced', classes=unique_classes, y=y_train)
class_weights = dict(zip(unique_classes, weights))
class_weights

{0: 0.7413793103448276,
 1: 0.2648870636550308,
 2: 16.125,
 3: 2.1147540983606556,
 4: 5.16,
 5: 6.7894736842105265}

# <span style="color:#e74c3c;"> CatBoost </span> Classifier


In [60]:
# creating pools for training and testing

train_pool = Pool(X_train, y_train, cat_features = cat_feat_idx)
test_pool = Pool(X_test, y_test, cat_features = cat_feat_idx)

In [61]:
# tuned with optima

model = CatBoostClassifier(iterations= 14400, task_type="GPU", devices='0:1', learning_rate =0.0029536992550707585 , min_data_in_leaf = 27 , class_weights=class_weights)

# model.fit(train_pool , verbose = 1000 )

## Swarm Intelligence

In [62]:
import pyswarms as ps

In [63]:
# Define the hyperparameter search space
hyperparameter_ranges = {
    'iterations': (1000, 5000),  # Reduce the range of boosting iterations
    'depth': (3, 10),           # Reduce the range of tree depth
    'learning_rate': (0.0001, 0.1),  # Reduce the learning rate range
    'min_data_in_leaf': (20, 40),
    'l2_leaf_reg': (1, 5),      # Reduce the range of regularization strength
}

In [69]:
def objective_function(x):
    # Round the elements of x to integers
    x = x.reshape(-1)  # Ensure x is a 1D array
    params = {
        'iterations': int(x[0]),
        'depth': int(x[1]),
        'learning_rate': float(x[2]),
        'min_data_in_leaf': int(x[3]),
        'l2_leaf_reg': int(x[4]),  
        'class_weights': class_weights,
        'task_type': 'GPU',
        'devices': '0:1',
        'cat_features': cat_feat_idx,
        'verbose': 0,
    }

    model = CatBoostClassifier(**params)
    model.fit(train_pool, verbose = 1000)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return -accuracy

In [70]:
n_particles = 50
n_dimensions = len(hyperparameter_ranges)
# bounds = (np.array([100, 3, 0.01, 1]), np.array([500, 8, 0.1, 5]))
bounds = (np.array([1000, 3, 0.0001, 10,1]), np.array([5000, 12, 0.1, 50, 20]))



optimizer = ps.single.GlobalBestPSO(n_particles=n_particles,
                                     dimensions=n_dimensions,
                                     bounds=bounds,
                                     options={'c1': 1.8, 'c2': 1.4, 'w': 0.6})


In [72]:
best_hyperparameters = optimizer.optimize(objective_function, iters=50)

2023-11-05 15:39:08,920 - pyswarms.single.global_best - INFO - Optimize for 50 iters with {'c1': 1.8, 'c2': 1.4, 'w': 0.6}
pyswarms.single.global_best:   0%|          |0/50


CatBoostError: C:/Go_Agent/pipelines/BuildMaster/catboost.git/catboost/private/libs/options/boosting_options.cpp:79: Learning rate should be non-zero

In [ ]:
len(hyperparameter_ranges)

In [ ]:
len(best_hyperparameters)

In [ ]:
best_hyperparameters

In [ ]:
best_accuracy, best_hyperparameter_values = best_hyperparameters

# Now, best_hyperparameter_values is an array of hyperparameter values
# You can print it to see the best hyperparameters
print("Best accuracy:", -best_accuracy)
print("Best hyperparameters:", best_hyperparameter_values)

In [ ]:
# Define the best hyperparameters as a dictionary
best_hyperparameters = {
    'iterations': best_hyperparameter_values[0],
    'depth': best_hyperparameter_values[1],
    'learning_rate': best_hyperparameter_values[2],
    'l2_leaf_reg': best_hyperparameter_values[3],
    'class_weights': class_weights,  # Assuming you have class_weights defined
    'task_type': 'GPU',
    'devices': '0:1',
    'cat_features': cat_feat_idx,
    'verbose': 0,
}

In [ ]:
# Create a CatBoost classifier with the best hyperparameters
model = CatBoostClassifier(**best_hyperparameters)

# Train the model on your training data
model.fit(train_pool , verbose = 1000 )

# <span style="color:#e74c3c;"> Results </span>


In [ ]:
# predictions and scores

pred = model.predict(test_pool)

r2_sr = r2_score(y_test, pred)
mse = mean_squared_error(y_test, pred)

print('R2 Score :{0:.5f}'.format(r2_sr))
print('Mean Squared Error :{0:.5f}'.format(mse))

In [ ]:
# classification report

clf_report = classification_report(pred, y_test )

print(clf_report)